# NSIDC DAAC Customize and Access Data Tutorial
### This tutorial will walk you though how to access NSIDC DAAC data using spatial and temporal filters, as well as how to request customization services including subsetting, reformatting, and reprojection. 

## Import packages


In [ ]:
import requests, getpass, socket, json, zipfile, io, math, os, shutil
from statistics import mean
from requests.auth import HTTPBasicAuth

## Create a token

### We will generate a token needed in order to access data using your Earthdata Login credentials, and we will apply that token to the following queries. If you do not already have an Earthdata Login account, go to http://urs.earthdata.nasa.gov to register. 

In [ ]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')
email = input('Email address associated with Earthdata Login account: ')


In [ ]:
# Request token from Common Metadata Repository using Earthdata credentials
token_api_url = 'https://api.echo.nasa.gov/echo-rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': uid,
        'password': pswd,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']
print(token)

# Back-up token in case of CMR downtime:

# token=9D0171BE-42AA-D88E-D14A-AA3449FBFCBA

## Select a data set of interest and determine the number and size of granules available within a time range and location.

### Let's begin discovering an NSIDC DAAC data set by first inputting the data set of interest and determining the most recent version number. We will also find out how many data granules exist over an area and time of interest. The Common Metadata Repository is queried to explore this information.  

In [ ]:
# Input data set short name (e.g. ATL03) of interest here.

short_name = input('Input short name, e.g. ATL03, here: ')

In [ ]:
# Get json response from CMR collection metadata

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)

### Now that we have the most recent version of this data set, let's determine the number of granules available over our area and time of interest.

In [ ]:
#Input bounding box

LL_lon = input('Input lower left longitude in decimal degrees: ')
LL_lat = input('Input lower left latitude in decimal degrees: ')
UR_lon = input('Input upper right longitude in decimal degrees: ')
UR_lat = input('Input upper right latitude in decimal degrees: ')

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat


In [ ]:
#Input temporal range 

start_date = input('Input start date in yyyy-MM-dd format: ')
start_time = input('Input start time in HH:mm:ss format: ')
end_date = input('Input end date in yyyy-MM-dd format: ')
end_time = input('Input end time in HH:mm:ss format: ')

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'


In [ ]:
# Query number of granules (paging over results)

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
params = {
    'short_name': short_name,
    'version': latest_version,
    'bounding_box': bounding_box,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1
}
granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')


### We will now query the average size of those granules as well as the total volume. 

In [ ]:
granule_sizes = [float(granule['granule_size']) for granule in granules]

print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')


### Although subsetting, reformatting, or reprojecting can alter the size of the granules, this "native" granule size can still be used to guide us towards the best download method to pursue, which we will come back to later on in this tutorial.

## Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

### The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request. 

### We will start by querying the service capability endpoint and gather service information that we will select in the next step.

In [ ]:
# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'
#capability_url = 'https://n5eil02u.ecs.nsidc.org/egi/capabilities/SPL3SMP.005.xml'

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

#collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

# reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

# SMAP-specific reprojection logic

#L1-L2 reprojection/reformatting options
if short_name == 'SPL1CTB' or 'SPL1CTB_E' or 'SPL2SMA' or 'SPL2SMP' or 'SPL2SMP_E' or 'SPL2SMAP': 
    format_proj = ['GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]
elif short_name == 'SPL2SMAP_S' or 'SPL3SMA' or 'SPL3SMP' or 'SPL3SMP_E' or 'SPL3SMAP' or 'SPL3FTA' or 'SPL3FTP' or 'SPL3FTP_E' or 'SPL4SMAU' or 'SPL4SMGP' or 'SPL4SMLM' or 'SPL4CMDL': 
    format_proj = ['No reformatting', 'GeoTIFF', 'NetCDF4-CF', 'HDF-EOS5']
    no_proj = [i for i in format_vals if i not in format_proj]

### We will now select subsetting, reformatting, and reprojection service options, if available.

In [ ]:
#print service information depending on service availability and select service options
    
if len(subagent) < 1 :
    print('No services exist for', short_name, 'version', latest_version)
    agent = 'NO'
    bbox = ''
    time = ''
    reformat = ''
    projection = ''
    projection_parameters = ''
    coverage = ''
else:
    agent = ''
    subdict = subagent[0]
    if subdict['spatialSubsetting'] == 'true':
        ss = input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
        if ss == 'y': bbox = bounding_box
        else: bbox = ''
    if subdict['temporalSubsetting'] == 'true':
        ts = input('Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n)')
        if ts == 'y': time = temporal 
        else: time = ''
    else: time = ''
    if len(format_vals) > 0 :
        print('These reformatting options are available:', format_vals)
        reformat = input('If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank.')
        # select reprojection options based on reformatting selection
        if reformat in format_proj and len(proj_vals) > 0 : 
            print('These reprojection options are available with your requested format:', proj_vals)
            projection = input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes, e.g. GEOGRAPHIC), otherwise leave blank.')
            # Enter required parameters for UTM North and South
            if projection == 'UTM NORTHERN HEMISPHERE' or projection == 'UTM SOUTHERN HEMISPHERE': 
                NZone = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                projection_parameters = str('NZone:' + NZone)
            else: projection_parameters = ''
        else: 
            print('No reprojection options are supported with your requested format')
            projection = ''
            projection_parameters = ''
    else: 
        reformat = ''
        projection = ''
        projection_parameters = ''


### Because variable subsetting can include a long list of variables to choose from, we will decide on variable subsetting separately from the service options above.

In [ ]:
# Select variable subsetting

import pprint

if len(variable_vals) > 0:
        v = input('Variable subsetting is available. Would you like to subset a selection of variables? (y/n)')
        if v == 'y':
            print('The', short_name, 'variables to select from include:')
            pprint.pprint(variable_vals)
            coverage = input('If you would like to subset by variable, copy and paste the variables you would like separated by comma. Make sure to omit quotes but include all forward slashes: ')
        else: coverage = ''

#no services selected
if reformat == '' and projection == '' and projection_parameters == '' and coverage == '' and time == '' and bbox == '':
    agent = 'NO'

## Request data from the NSIDC data access service.

### We will now set up our data download request. Recall that we queried the total number and volume of granules prior to applying customization services, so you can use these values to adjust the number of granules per request up to a limit of 100 granules. For now, let's select 10 granules to be processed in each zipped request. 

In [ ]:
# Determine how many individual orders we will request based on the number of granules requested

# Set number of granules requested per order, which we will initially set to 10.
page_size = 10
page_num = math.ceil(len(granules)/page_size)

#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

print('There will be', page_num, 'total order(s) processed for our', short_name, 'request.')

### Now, let's download the data directly to this notebook directory in a new Outputs folder. The progress of each order will be reported.

In [ ]:
# Create Outputs folder if folder does not already exist, request data service for each page number, and unzip outputs

path = str(os.getcwd() + '/Outputs')

if os.path.exists(path):
    for i in range(page_num):
        page_val = i + 1
        request_params = {'short_name': short_name, 'version': latest_version, 'temporal': temporal, 'time': time, 'bounding_box': bounding_box, 'bbox': bbox, 'format': reformat, 'projection': projection, 'projection_parameters': projection_parameters, 'Coverage': coverage, 'page_size': page_size, 'page_num': page_val, 'agent': agent, 'token': token, 'email': email, }
        print('Data request', page_val, 'is processing...')
        r = requests.get(base_url, params=request_params)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')
else:
    path = str(os.getcwd() + '/Outputs')
    os.mkdir(path)
    for i in range(page_num):
        page_val = i + 1
        request_params = {'short_name': short_name, 'version': latest_version, 'temporal': temporal, 'time': time, 'bounding_box': bounding_box, 'bbox': bbox, 'format': reformat, 'projection': projection, 'projection_parameters': projection_parameters, 'Coverage': coverage, 'page_size': page_size, 'page_num': page_val, 'agent': agent, 'token': token, 'email': email, }
        print('Data request', page_val, 'is processing...')
        r = requests.get(base_url, params=request_params)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')




### Finally, we will clean up the Output folder by removing individual order folders:

In [ ]:
#Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
        
for root, dirs, files in os.walk(path):
    for name in dirs:
        os.rmdir(os.path.join(root, name))

### To review, we have explored data availability and volume over a region and time of interest, discovered and selected data customization options, and downloaded data directly to our local machine. You are welcome to request different data sets, areas of interest, and/or customization services by re-running the notebook or starting again at the 'Select a data set of interest' step above. 